In [15]:
from autogenstudio import WorkflowManager

## AutoGen Studio Agent Workflow API Example

This notebook focuses on demonstrating capabilities of the autogen studio workflow python api.  

- Declarative Specification of an Agent Workflow 
- Loading the specification and running the resulting agent


> Note: The notebook currently demonstrates support for a two agent setup. Support for GroupChat is currently in development.

In [ ]:
# load workflow from json file
workflow_manager = WorkflowManager(workflow="two_agent.json")

# run the workflow on a task
task_query = "What is the height of the Eiffel Tower?. Dont write code, just respond to the question."
workflow_manager.run(message=task_query)

In [ ]:
# print the agent history
workflow_manager.agent_history

## Group Chat Agent

In [ ]:
# load workflow from json file
travel_workflow_manager = WorkflowManager(workflow="travel_groupchat.json")

# run the workflow on a task
task_query = "Plan a two day trip to Maui hawaii."
travel_workflow_manager.run(message=task_query)

In [ ]:
# print the agent history
print(len(travel_workflow_manager.agent_history), "agent messages were involved in the conversation")
travel_workflow_manager.agent_history